In [1]:
from sklearn.metrics import (
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score
)

In [6]:
import pandas as  pd

df = pd.read_csv('D:/practica/5 - Введение в машинное обучение/datasets/users_behavior.csv')


In [7]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [9]:
tarif_plan = df.drop(['is_ultra'], axis = 1)
target = df['is_ultra']
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [10]:
from sklearn.model_selection import train_test_split
tarif_plan_train, tarif_plan_test, target_train, target_test = train_test_split(tarif_plan, target, test_size = 0.3, random_state=12345)


In [11]:
tarif_plan_train

,calls,minutes,messages,mb_used
372,100.0,626.94,43.0,22690.35
703,28.0,195.36,29.0,11179.93
2406,53.0,359.13,0.0,13758.39
2891,58.0,467.65,0.0,19171.85
1850,144.0,931.29,42.0,30250.66
...,...,...,...,...
2817,12.0,86.62,22.0,36628.85
546,65.0,458.46,0.0,15214.25
382,144.0,906.18,0.0,25002.44
2177,38.0,301.27,37.0,28914.24


In [12]:
df.is_ultra.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [13]:
def calc_metrics(prefix, model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)
    y_prob_one = y_prob[:, 1]
    auc_roc = roc_auc_score(y_test, y_prob_one)
    
    print(prefix, 'precision:', precision_score(y_test, y_pred))
    print(prefix, 'recall:', recall_score(y_test, y_pred))
    print(prefix, 'f1:', f1_score(y_test, y_pred))
    print(prefix, 'AUC-ROC:', auc_roc)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

best_model = None
best_accuracy = 0


for est in range(1, 15):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(tarif_plan_train, target_train)
    
    result = model.score(tarif_plan_test, target_test)
    if result > best_accuracy:
        best_model = model
        best_accuracy = result
        
print("best random forest accuracy on valid:", best_accuracy)

best random forest accuracy on valid: 0.7823834196891192


In [15]:
test_score = best_model.score(tarif_plan_test, target_test)

print('accuracy on test:', test_score)


accuracy on test: 0.7823834196891192


In [32]:
calc_metrics('RF', model, tarif_plan_train, target_train)

RF precision: 0.9970238095238095
RF recall: 0.9696092619392185
RF f1: 0.9831254585473221
RF AUC-ROC: 0.9997557074359684


In [39]:
import numpy as np

for itr_num in range(1, 6):
    sanity_test_target = np.random.choice([0, 1], size=len(target_test))
    sanity_check_score = best_model.score(tarif_plan_test, sanity_test_target)
    print('sanity check score:', sanity_check_score, 'on iteration:', itr_num)

sanity check score: 0.47925311203319504 on iteration: 1
sanity check score: 0.508298755186722 on iteration: 2
sanity check score: 0.4896265560165975 on iteration: 3
sanity check score: 0.475103734439834 on iteration: 4
sanity check score: 0.5228215767634855 on iteration: 5


In [40]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [46]:
model = RandomForestClassifier()
scores = cross_val_score(model, tarif_plan_test, target_test, cv=5, scoring='accuracy')
print('Средняя точность:', scores.mean())
print('Стандартное отклонение:', scores.std())

Средняя точность: 0.783419689119171
Стандартное отклонение: 0.007614994019015089
